In [4]:
import json
import pandas as pd
import networkx as nx
import folium
from scipy.spatial import KDTree

# --- 1. 데이터 로드 및 기본 설정 (기존과 동일) ---
with open('../../../../data/04_final_data/final_edges.geojson', encoding='utf-8') as f:
    edges_geo = json.load(f)
nodes_df = pd.read_csv('../../../../data/04_final_data/final_nodes.csv')
tree_char_df = pd.read_csv('../../../../data/02_intermediate/tree-characteristics.csv')

def get_seasonal_trees(season):
    return set(tree_char_df[tree_char_df[season] == 1]['수목명'])

node_pos = {row['osmid']: (row['lat'], row['lon']) for _, row in nodes_df.iterrows()}

G = nx.Graph()
for feature in edges_geo['features']:
    props = feature['properties']
    u, v = props['u'], props['v']
    length = props.get('length', 1)
    type_keys = ['park', 'mountain', 'river', 'tree-line', 'road']
    edge_type = next((k for k in type_keys if props.get(k, 0) == 1), 'road')
    tree_names = props.get('tree', '').split()
    G.add_edge(u, v, length=length, type=edge_type, tree=tree_names)

def filter_graph_by_bbox_and_type(G, node_pos, start_lat, start_lon, walk_km, bbox_km=2.0):
    lat_margin = bbox_km / 111.0
    lon_margin = bbox_km / 88.0
    min_lat, max_lat = start_lat - lat_margin, start_lat + lat_margin
    min_lon, max_lon = start_lon - lon_margin, start_lon + lon_margin
    nodes_in_bbox = [osmid for osmid, (lat, lon) in node_pos.items() if min_lat <= lat <= max_lat and min_lon <= lon <= max_lon]
    subG = G.subgraph(nodes_in_bbox).copy()
    return subG

def _get_preference_score(data, seasonal_trees):
    type_weight_map = {
        'park': 0.3, 'river': 0.3, 'mountain': 0.3,
        'tree_line': 0.5, 'road': 2.0
    }
    type_modifier = type_weight_map.get(data.get('type'), 1.5)
    tree_modifier = 0.7 if any(t in seasonal_trees for t in data.get('tree', [])) else 1.0
    return type_modifier * tree_modifier

def edge_weight(u, v, season, G):
    data = G[u][v]
    seasonal_trees = get_seasonal_trees(season)
    preference_score = _get_preference_score(data, seasonal_trees)
    return preference_score * data['length']

# --- 2. [수정] 메인 추천 알고리즘 (편도 경로 추천으로 변경) ---
def recommend_path(G, node_pos, start_lat, start_lon, season, total_walk_km, tolerance=0.2):
    # 시작점 노드 찾기
    node_coords = list(node_pos.values())
    node_osmids = list(node_pos.keys())
    kdtree = KDTree(node_coords)
    _, start_node_idx = kdtree.query([start_lat, start_lon])
    start_osmid = node_osmids[start_node_idx]

    # 서브그래프 생성
    subG = filter_graph_by_bbox_and_type(G, node_pos, start_lat, start_lon, total_walk_km)
    if not subG.has_node(start_osmid):
        print("오류: 시작점 주변에 탐색할 경로가 없습니다.")
        return None
        
    # 경로 탐색용 가중치 적용
    for u, v in subG.edges():
        subG[u][v]['weight'] = edge_weight(u, v, season, subG)

    # 매력적인 POI 찾기
    edge_attractiveness = []
    seasonal_trees = get_seasonal_trees(season)
    for u, v, data in subG.edges(data=True):
        preference_score = _get_preference_score(data, seasonal_trees)
        attractiveness_score = 1 / preference_score
        edge_attractiveness.append(((u, v), attractiveness_score))

    edge_attractiveness.sort(key=lambda x: x[1], reverse=True)
    top_10_percent_index = int(len(edge_attractiveness) * 0.1)
    
    if top_10_percent_index > 0:
        top_edges = [edge for edge, score in edge_attractiveness[:top_10_percent_index]]
        pois = {n for u, v in top_edges for n in (u, v)}
    else:
        pois = {n for u, v, data in subG.edges(data=True) if data.get('type') != 'road' for n in (u,v)}

    if not pois:
        print("알림: 주변에 추천할 만한 경로가 없습니다.")
        return None

    # 다익스트라 경로 계산
    try:
        path_lengths = nx.single_source_dijkstra_path_length(subG, start_osmid, weight='length')
    except nx.NetworkXNoPath:
        print(f"오류: 시작점 {start_osmid}이(가) 주변 경로와 연결되어 있지 않습니다.")
        return None

    # [수정] 목표 편도 거리에 맞는 POI 후보군 필터링
    candidates = []
    target_one_way_m = total_walk_km * 1000 / 2  # 목표 거리를 절반으로 설정
    for poi in pois:
        if poi in path_lengths:
            path_length_to_poi = path_lengths[poi]
            # 편도 거리를 기준으로 비교
            if abs(path_length_to_poi - target_one_way_m) <= target_one_way_m * tolerance:
                candidates.append((poi, path_length_to_poi))
    
    if not candidates:
        print(f"{total_walk_km}km 왕복 거리에 맞는 추천 경로를 찾지 못했습니다.")
        return None
        
    # [수정] 최적 경로 선택 (편도 기준)
    best_poi, _ = min(candidates, key=lambda x: abs(x[1] - target_one_way_m))
    
    # [수정] 최종 경로 생성 (편도)
    final_path = nx.shortest_path(subG, start_osmid, best_poi, weight='weight')
    
    return final_path

# --- 3. [수정] 함수 호출 및 시각화 ---
start_lat, start_lon = 37.567893, 127.051643 
walk_km = 5  # 총 왕복 산책 거리
season = '봄'

# 수정된 함수 호출
path_nodes = recommend_path(G, node_pos, start_lat, start_lon, season, walk_km)

if path_nodes:
    # 편도 경로 길이 계산
    one_way_length = sum(G.edges[path_nodes[i], path_nodes[i+1]]['length'] for i in range(len(path_nodes)-1))
    
    # 수정된 안내 문구
    print(f"--- 편도 경로 추천 ---")
    print(f"편도 거리: {one_way_length/1000:.2f} km")
    print(f"왕복 시 예상 거리: {one_way_length*2/1000:.2f} km")
    
    m = folium.Map(location=[start_lat, start_lon], zoom_start=15)
    folium.Marker(location=[start_lat, start_lon], popup="시작점", icon=folium.Icon(color='red')).add_to(m)
    
    # 도착점(POI)에 마커 추가
    end_pos = node_pos[path_nodes[-1]]
    folium.Marker(location=end_pos, popup="반환점", icon=folium.Icon(color='blue')).add_to(m)

    path_coords = [node_pos[node] for node in path_nodes]
    folium.PolyLine(locations=path_coords, color='purple', weight=5, opacity=0.8).add_to(m)
    
    m.save('recommended_walk_path.html')
    print("\n'recommended_walk_path.html' 파일이 저장되었습니다.")
else:
    print('경로를 찾지 못하여 지도를 생성하지 않았습니다.')


--- 편도 경로 추천 ---
편도 거리: 2.67 km
왕복 시 예상 거리: 5.33 km

'recommended_walk_path.html' 파일이 저장되었습니다.
